In [1]:
import os
import gc
import sys
import cv2
import json
import argparse
import logging
import time
from tqdm import tqdm

from skimage.color import rgb2hsv
from skimage.filters import threshold_otsu
import matplotlib.pyplot as plt
import numpy as np
import openslide

from multiprocessing import Pool, Value, Lock

In [3]:
def process(opts):
    anno_num, i, wsi_name, x_center, y_center, slide, level, patch_size, patch_path  = opts
    x = x_center # int(int(x_center) - patch_size / 2)
    y = y_center #int(int(y_center) - patch_size / 2)
    
    if x < 0 or y < 0:
        return 
    print((x, y))
    img = slide.read_region(
        (x, y), level,
        (patch_size, patch_size)).convert('RGB')

    img.save(os.path.join(patch_path, wsi_name + '_anno_' + str(anno_num) + '_' + str(i) + '.png'))

In [5]:
class patch_point_in_mask_gen(object):
    '''
    extract centre point from mask
    inputs: mask path, centre point number
    outputs: centre point
    '''

    def __init__(self, vertices, padding=256):
        print('vertices', vertices)
        self.vertices = vertices #np.array([np.array(point) for point in vertices])
        self.padding = padding
        
    def get_patch_point(self):
        
        start_x, start_y = min(self.vertices[:, 0]), min(self.vertices[:, 1])
        end_x, end_y = max(self.vertices[:, 0]), max(self.vertices[:, 1])
        centre_points = []
        x_range = [int(xx) for xx in range(start_x, end_x-256, self.padding)]
        y_range = [int(yy) for yy in range(start_y, end_y-256, self.padding)]
        
        for x in x_range:
            for y in y_range:
                if cv2.pointPolygonTest(self.vertices, (x, y), measureDist=False):
                    #print([x,y])
                    centre_points.append([x,y])
        centre_points = np.array(centre_points)
        print(np.array(centre_points).shape)
        
        return centre_points

In [6]:
def get_tissue_mask(wsi_path, level):
    
    patch_size=512
    json_path = wsi_path.replace('svs', 'json')
    wsi_name = os.path.basename(wsi_path).replace('.svs', '')
    print(wsi_name)
    patch_path = os.path.join('/homeb/junjianli/data/competition/jiangsu-seed/tmp/patch', wsi_name)
    
    if not os.path.exists(patch_path):
        os.makedirs(patch_path, exist_ok=True)
        
    if not os.path.exists(json_path):
        return 
        
    slide = openslide.OpenSlide(wsi_path)
    w, h = slide.level_dimensions[level]
    print(f'w, h:{w, h}')
    # get the factor of level * e.g. level 6 is 2^6
    factor = slide.level_downsamples[level]
    print(f'factor:{factor}')
    
    with open(json_path) as f:
        print('load json')
        dicts = json.load(f)
        
    tumor_polygons = dicts['contexts']
    for polygon_num, tumor_polygon in enumerate(tumor_polygons):
        # plot a polygon
        name = tumor_polygon["name"]
        p = []
        for point in tumor_polygon['points']:
            x, y = point.get('x'), point.get('y')
            p.append([x if x>0 else 0, y if y>0 else 0])
        print(p)
        vertices = np.array(p) / factor
        vertices = vertices.astype(np.int32)
        
        sampled_points = patch_point_in_mask_gen(vertices, padding=256).get_patch_point()
        sampled_points = (sampled_points * factor).astype(np.int32) # make sure the factor

        opts_list = [('anno_' + str(polygon_num), point_num, wsi_name, point[0], point[1], slide, level, patch_size, patch_path) \
                     for point_num, point in enumerate(sampled_points)]
        for opt in tqdm(opts_list):
            process(opt)
#         pool = Pool(processes=0)
#         pool.map(process, opts_list)
#         pool.close()
#         pool.join()

In [ ]:
get_tissue_mask(wsi_path, level)

In [7]:
if __name__ == "__main__":
    wsi_path='/homeb/junjianli/data/competition/jiangsu-seed/train/T3/YxRM16.svs'
    get_tissue_mask(wsi_path, 2)

YxRM16
w, h:(2125, 1920)
factor:16.0
load json
{'contexts': [{'name': 'tumor', 'parent': -1, 'points': [{'x': 8903.3779296875, 'y': 1994.356689453125}, {'x': 9686.875, 'y': 2492.94580078125}, {'x': 10577.212890625, 'y': 2920.30810546875}, {'x': 11218.2568359375, 'y': 3276.443115234375}, {'x': 12001.75390625, 'y': 3205.216064453125}, {'x': 12714.0234375, 'y': 3347.670166015625}, {'x': 13319.4541015625, 'y': 3810.645751953125}, {'x': 14138.564453125, 'y': 3917.486328125}, {'x': 14850.8349609375, 'y': 4380.4619140625}, {'x': 15563.1044921875, 'y': 4558.52978515625}, {'x': 16275.375, 'y': 4772.21044921875}, {'x': 16987.64453125, 'y': 5128.345703125}, {'x': 17771.142578125, 'y': 5306.41357421875}, {'x': 18483.412109375, 'y': 5520.09423828125}, {'x': 19266.91015625, 'y': 5698.162109375}, {'x': 19907.953125, 'y': 6018.68359375}, {'x': 20620.224609375, 'y': 6339.205078125}, {'x': 21261.267578125, 'y': 6695.34033203125}, {'x': 21973.537109375, 'y': 6873.40771484375}, {'x': 22757.03515625, 'y': 

 12%|█▎        | 3/24 [00:00<00:00, 28.66it/s]

(7328, 1984)
(7328, 6080)
(7328, 10176)
(7328, 14272)
(11424, 1984)
(11424, 6080)
(11424, 10176)


 46%|████▌     | 11/24 [00:00<00:00, 28.38it/s]

(11424, 14272)
(15520, 1984)
(15520, 6080)
(15520, 10176)
(15520, 14272)
(19616, 1984)


...

(13568, 21408)
(13568, 25504)
(17664, 9120)
(17664, 13216)
(17664, 17312)
(17664, 21408)
(17664, 25504)


 88%|████████▊ | 30/34 [00:00<00:00, 36.45it/s]

(21760, 9120)
(21760, 13216)
(21760, 17312)
(21760, 21408)
(21760, 25504)
(25856, 9120)
(25856, 13216)
(25856, 17312)


100%|██████████| 34/34 [00:00<00:00, 36.39it/s]

(25856, 21408)
(25856, 25504)
